<a href="https://colab.research.google.com/github/pascal-maker/machinelearning/blob/main/Hyperparameter_Optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Hyperparameter optimization by means of cross-validation

In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import linear_model
from sklearn import svm

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn import preprocessing
from sklearn.metrics import confusion_matrix, accuracy_score
import seaborn as sns

In [2]:
dataset = pd.read_csv('diabetes.csv')
dataset.head(10)

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,92,62,32,126,32.0,85.0,46,0
1,2,125,60,20,140,33.8,88.0,31,0
2,0,117,80,31,53,45.2,89.0,24,0
3,1,87,78,27,32,34.6,101.0,22,0
4,3,116,74,15,105,26.3,107.0,24,0
5,1,128,82,17,183,27.5,115.0,22,0
6,4,110,76,20,100,28.4,118.0,27,0
7,6,105,70,32,68,30.8,122.0,37,0
8,1,157,72,21,168,25.6,123.0,24,0
9,2,102,86,36,120,45.5,127.0,23,1


In [3]:
y = dataset['Outcome'].values
X = dataset.drop('Outcome',axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 100, random_state =0)
print(X_train.shape)

scaler = preprocessing.StandardScaler().fit(X_train)  # Normaliseer naar gemiddelde = 0 en standaardafwijking = 1

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

(280, 8)


Features (X): Alle kolommen behalve Outcome.

Targets (y): Alleen de Outcome kolom.

Train/Test Split: 100 samples voor de test set, de rest voor de train set.

Waarom random_state? Voor reproduceerbare resultaten.



## Grid Search

In [4]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

model = SVC()
paramaters = [
        {'kernel': ['linear'], 'C': np.linspace(0.01,10000,10)}, # 10 models
        {'kernel': ['rbf'], 'C': np.linspace(0.01,10000,10), 'gamma': [0.0001, 0.001, 0.01, 0.1, 0.2]}, #10x5 = 50 models
        {'kernel': ['poly'], 'C':np.linspace(0.01,10000,10)} ] # 10 models
grid_search = GridSearchCV(estimator = model,
                           param_grid = paramaters,
                           scoring = 'accuracy',
                           cv = 5, # k=5
                           n_jobs = -1,
                           verbose =5)
grid_search = grid_search.fit(X_train, y_train)

best_accuracy = grid_search.best_score_
best_parameters = grid_search.best_params_

print('Best accuracy : ', grid_search.best_score_)
print('Best parameters :', grid_search.best_params_  )

y_pred = grid_search.predict(X_test)
print(classification_report(y_test, y_pred))

cf = confusion_matrix(y_test, y_pred)
print(cf)
print(accuracy_score(y_test, y_pred) * 100)

Fitting 5 folds for each of 70 candidates, totalling 350 fits
Best accuracy :  0.7928571428571429
Best parameters : {'C': np.float64(1111.12), 'gamma': 0.0001, 'kernel': 'rbf'}
              precision    recall  f1-score   support

           0       0.74      0.89      0.81        63
           1       0.71      0.46      0.56        37

    accuracy                           0.73       100
   macro avg       0.72      0.67      0.68       100
weighted avg       0.73      0.73      0.71       100

[[56  7]
 [20 17]]
73.0


predict(): Voorspel uitkomsten voor de test set met de beste hyperparameters.

classification_report(): Precision, Recall, F1-Score.

confusion_matrix(): Laat zien hoeveel samples correct en incorrect zijn geclassificeerd.

accuracy_score(): Percentage correcte voorspellingen.

Grid Search: Test alle combinaties van hyperparameters.

Param Grid:

Linear Kernel: Alleen C (10 waarden).

RBF Kernel: C (10 waarden) en gamma (5 waarden).

Polynomial Kernel: Alleen C (10 waarden).

Aantal Geteste Modellen:

10 + 50 + 10 = 70 modellen.

Cross-Validation: 5-fold CV voor betrouwbaardere resultaten.

n_jobs = -1: Gebruikt alle CPU cores.



best_score_: Hoogste cross-validated accuracy.

best_params_: Beste combinatie van hyperparameters.

## Random search

In [6]:

from sklearn.model_selection import RandomizedSearchCV
# distributions
from scipy.stats import randint
from scipy.stats import uniform



model = SVC()
parameters = {'kernel': ['linear','rbf','poly'],
              'C': uniform(0.001, 10000), # haal C uit een random uniform distribution
              'gamma': uniform(0.001, 0.2)}


n_iter_search = 10


random_search = RandomizedSearchCV(model, param_distributions=parameters,cv=5,n_iter=n_iter_search,n_jobs = -1,verbose=1)

random_search = random_search.fit(X_train, y_train)

best_accuracy = random_search.best_score_
best_parameters = random_search.best_params_

print('Best accuracy : ', random_search.best_score_)
print('Best parameters :',random_search.best_params_  )

y_pred = random_search.predict(X_test)
print(classification_report(y_test, y_pred))

cf = confusion_matrix(y_test, y_pred)
print(cf)
print(accuracy_score(y_test, y_pred) * 100)


Fitting 5 folds for each of 10 candidates, totalling 50 fits
Best accuracy :  0.7821428571428571
Best parameters : {'C': np.float64(9997.16945693932), 'gamma': np.float64(0.056301329422715665), 'kernel': 'linear'}
              precision    recall  f1-score   support

           0       0.74      0.84      0.79        63
           1       0.64      0.49      0.55        37

    accuracy                           0.71       100
   macro avg       0.69      0.66      0.67       100
weighted avg       0.70      0.71      0.70       100

[[53 10]
 [19 18]]
71.0


Randomized Search: Test random combinaties van hyperparameters.

Uniforme Distributie: Maakt het mogelijk om een breed bereik aan C en gamma te testen.

n_iter_search: Test slechts 10 combinaties in plaats van 70 (sneller).

Voordeel: Efficiënter dan Grid Search bij grote parameter ruimtes.Uitvoer: Geeft de beste combinatie van parameters en de bijbehorende accuracy.

Vergelijk: Hoe presteert dit model in vergelijking met Grid Search?



## Bayes search

In [9]:
# BayesSearchCV

from skopt import BayesSearchCV #scikit-optimize

# distributions
from scipy.stats import randint
from scipy.stats import uniform


model = SVC()
parameters = {'kernel': ['linear','rbf','poly'],
              'C': (0.01, 100,'uniform'),
              'gamma': (0.001, 0.2,'uniform')}

n_iter_search = 10

Bayes_search = BayesSearchCV(model,parameters,n_iter=n_iter_search,cv=5,verbose=1)


Bayes_search.fit(X_train, y_train)

best_accuracy = Bayes_search.best_score_
best_parameters = Bayes_search.best_params_

print('Best accuracy : ', Bayes_search.best_score_)
print('Best parameters :',Bayes_search.best_params_  )

y_pred = Bayes_search.predict(X_test)
print(classification_report(y_test, y_pred))

cf = confusion_matrix(y_test, y_pred)
print(cf)
print(accuracy_score(y_test, y_pred) * 100)

Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Best accuracy :  0.7821428571428571
Best parameters : OrderedDict([('C', 18.13335784850457), ('gamma', 0.02560094087938019), ('kernel', 'linear')])
              precision    recall  f1-score   support

           0       0.74      0.84      0.79        63
           1       0.64      0.49      0.55        37

    accuracy                           0.71       100
   macro avg       0.69      0.66      0.67 

Bayesian Search: Geavanceerdere zoekstrategie die sneller convergeert naar optimale combinaties.

n_iter_search: Test 10 combinaties.

Voordeel: Efficiënter dan zowel Grid Search als Randomized Search Uitvoer: Geeft de beste combinatie van parameters en de bijbehorende accuracy.

Vergelijk: Hoe presteert dit model in vergelijking met Grid Search en Randomized Search?



In [8]:

!pip install scikit-optimize


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.8/107.8 kB 3.2 MB/s eta 0:00:00
